# Interpretable Predictions of Tree-based Ensembles via Actionable Feature Tweaking【KDD 2017】

This ipynb is inspired by the following link:  
http://setten-qb.hatenablog.com/entry/2017/10/22/232016

I fixed some codes and added some explanations:
1. Fix `load_iris()` to `datasets.load_iris()` at In [2]
1. Fix `rfc.fit(x, y)` to `rfc.fit(x_arr, y_arr)` at In [3]
1. Fix `aim_label = 3` to `aim_label = 2` at In [7] and [22]
1. Add the usage of feature_tweaking()
1. Add featureTweakPy.py to extract functions

## Package Import

In [1]:
import numpy as np
import scipy.stats
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import copy

## Dataset Import

In [2]:
iris = datasets.load_iris()

x_arr = iris['data']
mean_x = x_arr.mean(axis=0)
std_x = x_arr.std(axis=0)
x_arr = scipy.stats.zscore(x_arr)
y_arr = iris['target']

## Creation of Function

In [3]:
rfc = RandomForestClassifier()
rfc.fit(x_arr, y_arr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

`rfc` contains infomation of each weak classifier. For example `wc = rfc[0]` contains the following one.

In [4]:
wc = rfc[0]
wc

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=1463721801, splitter='best')

This is information of left child node.

In [5]:
children_left = wc.tree_.children_left
children_left

array([ 1, -1,  3,  4, -1,  6,  7, -1,  9, -1, -1, 12, 13, 14, -1, 16, -1,
       -1, -1, -1, 21, 22, -1, -1, -1])

Here, `-1` means the node doesn't have a children left node.

In [6]:
leaf_nodes = np.where(children_left == -1)[0]
leaf_nodes

array([ 1,  4,  7,  9, 10, 14, 16, 17, 18, 19, 22, 23, 24])

Now, we want the information of paths that predict positive label. We regard `2` as a positive label.

In [7]:
class_labels = [0, 1, 2]
aim_label = 2

In [8]:
leaf_values = wc.tree_.value[leaf_nodes].reshape(len(leaf_nodes), len(class_labels))
leaf_values

array([[43.,  0.,  0.],
       [ 0., 41.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  4.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., 14.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  6.],
       [ 0.,  0., 35.]])

In [9]:
leaf_nodes = np.where(leaf_values[:, aim_label] != 0)[0]
leaf_nodes

array([ 2,  3,  7,  9, 11, 12])

We use one leaf node as a sample.

In [10]:
leaf_node = leaf_nodes[0]

Then, let's identify the path from this node to the root node.

In [11]:
children_right = wc.tree_.children_right
children_right

array([ 2, -1, 20,  5, -1, 11,  8, -1, 10, -1, -1, 19, 18, 15, -1, 17, -1,
       -1, -1, -1, 24, 23, -1, -1, -1])

In [12]:
feature = wc.tree_.feature
feature

array([ 3, -2,  3,  3, -2,  1,  3, -2,  0, -2, -2,  0,  1,  1, -2,  2, -2,
       -2, -2, -2,  0,  2, -2, -2, -2])

In [13]:
threshold = wc.tree_.threshold
threshold

array([-0.52413893, -2.        ,  0.72485435,  0.19896244, -2.        ,
       -0.93486798,  0.3304354 , -2.        , -0.29484183, -2.        ,
       -2.        ,  1.4620924 , -0.47206205, -0.70346498, -2.        ,
        0.67746007, -2.        , -2.        , -2.        , -2.        ,
        0.2504136 ,  0.62059438, -2.        , -2.        , -2.        ])

In [14]:
child_node = leaf_node
parent_node = -100  # initialize
parents_left = []
parents_right = []
while (parent_node != 0):
    if (np.where(children_left == child_node)[0].shape == (0, )):
        parent_left = -1
        parent_right = np.where(
            children_right == child_node)[0][0]
        parent_node = parent_right
    elif (np.where(children_right == child_node)[0].shape == (0, )):
        parent_right = -1
        parent_left = np.where(children_left == child_node)[0][0]
        parent_node = parent_left
    parents_left.append(parent_left)
    parents_right.append(parent_right)
    """ proccess for next step """
    child_node = parent_node

In [15]:
""" search the path to the selected leaf node """
paths = {}
for leaf_node in leaf_nodes:
    """ correspond leaf node to left and right parents """
    child_node = leaf_node
    parent_node = -100  # initialize
    parents_left = [] 
    parents_right = [] 
    while (parent_node != 0):
        if (np.where(children_left == child_node)[0].shape == (0, )):
            parent_left = -1
            parent_right = np.where(
                children_right == child_node)[0][0]
            parent_node = parent_right
        elif (np.where(children_right == child_node)[0].shape == (0, )):
            parent_right = -1
            parent_left = np.where(children_left == child_node)[0][0]
            parent_node = parent_left
        parents_left.append(parent_left)
        parents_right.append(parent_right)
        """ proccess for next step """
        child_node = parent_node
    # nodes dictionary containing left parents and right parents
    paths[leaf_node] = (parents_left, parents_right)

In [16]:
paths

{2: ([-1], [0]),
 3: ([2, -1], [-1, 0]),
 7: ([6, 5, -1, 2, -1], [-1, -1, 3, -1, 0]),
 9: ([8, -1, 5, -1, 2, -1], [-1, 6, -1, 3, -1, 0]),
 11: ([-1, -1, 2, -1], [5, 3, -1, 0]),
 12: ([11, -1, -1, 2, -1], [-1, 5, 3, -1, 0])}

Next, we extract information about branch conditions at the nodes in the path.
- features
- thresholds
- inequality_symbols

In [17]:
path_info = {}
for i in paths:
    node_ids = []  # node ids used in the current node
    # inequality symbols used in the current node
    inequality_symbols = []
    thresholds = []  # thretholds used in the current node
    features = []  # features used in the current node
    parents_left, parents_right = paths[i]
    for idx in range(len(parents_left)):
        if (parents_left[idx] != -1):
            """ the child node is the left child of the parent """
            node_id = parents_left[idx]  # node id
            node_ids.append(node_id)
            inequality_symbols.append(0)
            thresholds.append(threshold[node_id])
            features.append(feature[node_id])
        elif (parents_right[idx] != -1):
            """ the child node is the right child of the parent """
            node_id = parents_right[idx]
            node_ids.append(node_id)
            inequality_symbols.append(1)
            thresholds.append(threshold[node_id])
            features.append(feature[node_id])
        path_info[i] = {'node_id': node_ids,
                        'inequality_symbol': inequality_symbols,
                        'threshold': thresholds,
                        'feature': features}

In [18]:
path_info

{2: {'node_id': [0],
  'inequality_symbol': [1],
  'threshold': [-0.5241389274597168],
  'feature': [3]},
 3: {'node_id': [2, 0],
  'inequality_symbol': [0, 1],
  'threshold': [0.7248543500900269, -0.5241389274597168],
  'feature': [3, 3]},
 7: {'node_id': [6, 5, 3, 2, 0],
  'inequality_symbol': [0, 0, 1, 0, 1],
  'threshold': [0.3304353952407837,
   -0.9348679780960083,
   0.19896243512630463,
   0.7248543500900269,
   -0.5241389274597168],
  'feature': [3, 1, 3, 3, 3]},
 9: {'node_id': [8, 6, 5, 3, 2, 0],
  'inequality_symbol': [0, 1, 0, 1, 0, 1],
  'threshold': [-0.29484182596206665,
   0.3304353952407837,
   -0.9348679780960083,
   0.19896243512630463,
   0.7248543500900269,
   -0.5241389274597168],
  'feature': [0, 3, 1, 3, 3, 3]},
 11: {'node_id': [5, 3, 2, 0],
  'inequality_symbol': [1, 1, 0, 1],
  'threshold': [-0.9348679780960083,
   0.19896243512630463,
   0.7248543500900269,
   -0.5241389274597168],
  'feature': [1, 3, 3, 3]},
 12: {'node_id': [11, 5, 3, 2, 0],
  'inequality

Here, the proccess is functionized as follows:

In [19]:
def search_path(estimator, class_labels, aim_label):
    """
    return path index list containing [{leaf node id, inequality symbol, threshold, feature index}].
    estimator: decision tree
    maxj: the number of selected leaf nodes
    """
    """ select leaf nodes whose outcome is aim_label """
    children_left = estimator.tree_.children_left  # information of left child node
    children_right = estimator.tree_.children_right
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    # leaf nodes ID
    leaf_nodes = np.where(children_left == -1)[0]
    # outcomes of leaf nodes
    leaf_values = estimator.tree_.value[leaf_nodes].reshape(len(leaf_nodes), len(class_labels))
    # select the leaf nodes whose outcome is aim_label
    leaf_nodes = np.where(leaf_values[:, aim_label] != 0)[0]
    """ search the path to the selected leaf node """
    paths = {}
    for leaf_node in leaf_nodes:
        """ correspond leaf node to left and right parents """
        child_node = leaf_node
        parent_node = -100  # initialize
        parents_left = [] 
        parents_right = [] 
        while (parent_node != 0):
            if (np.where(children_left == child_node)[0].shape == (0, )):
                parent_left = -1
                parent_right = np.where(
                    children_right == child_node)[0][0]
                parent_node = parent_right
            elif (np.where(children_right == child_node)[0].shape == (0, )):
                parent_right = -1
                parent_left = np.where(children_left == child_node)[0][0]
                parent_node = parent_left
            parents_left.append(parent_left)
            parents_right.append(parent_right)
            """ for next step """
            child_node = parent_node
        # nodes dictionary containing left parents and right parents
        paths[leaf_node] = (parents_left, parents_right)
        
    path_info = {}
    for i in paths:
        node_ids = []  # node ids used in the current node
        # inequality symbols used in the current node
        inequality_symbols = []
        thresholds = []  # thretholds used in the current node
        features = []  # features used in the current node
        parents_left, parents_right = paths[i]
        for idx in range(len(parents_left)):
            if (parents_left[idx] != -1):
                """ the child node is the left child of the parent """
                node_id = parents_left[idx]  # node id
                node_ids.append(node_id)
                inequality_symbols.append(0)
                thresholds.append(threshold[node_id])
                features.append(feature[node_id])
            elif (parents_right[idx] != -1):
                """ the child node is the right child of the parent """
                node_id = parents_right[idx]
                node_ids.append(node_id)
                inequality_symbols.append(1)
                thresholds.append(threshold[node_id])
                features.append(feature[node_id])
            path_info[i] = {'node_id': node_ids,
                            'inequality_symbol': inequality_symbols,
                            'threshold': thresholds,
                            'feature': features}
    return path_info

Next step is the culculation of ε-satisfactory instance

In [20]:
def esatisfactory_instance(x, epsilon, path_info):
    """
    return the epsilon satisfactory instance of x.
    """
    esatisfactory = copy.deepcopy(x)
    for i in range(len(path_info['feature'])):
        # feature index
        feature_idx = path_info['feature'][i]
        # threshold used in the current node
        threshold_value = path_info['threshold'][i]
        # inequality symbol
        inequality_symbol = path_info['inequality_symbol'][i]
        if inequality_symbol == 0:
            esatisfactory[feature_idx] = threshold_value - epsilon
        elif inequality_symbol == 1:
            esatisfactory[feature_idx] = threshold_value + epsilon
        else:
            print('something wrong')
    return esatisfactory

Finally, the follwing is the implementation of the proposed method using 2 functions.

In [21]:
def feature_tweaking(ensemble_classifier, x, class_labels, aim_label, epsilon, cost_func):
    """
    This function return the active feature tweaking vector.
    x: feature vector
    class_labels: list containing the all class labels
    aim_label: the label which we want to transform the label of x to
    """
    """ initialize """
    x_out = copy.deepcopy(x)  # initialize output
    delta_mini = 10**3  # initialize cost
    for estimator in ensemble_classifier:
        if (ensemble_classifier.predict(x.reshape(1, -1)) == estimator.predict(x.reshape(1, -1))
            and estimator.predict(x.reshape(1, -1) != aim_label)):
            paths_info = search_path(estimator, class_labels, aim_label)
            for key in paths_info:
                """ generate epsilon-satisfactory instance """
                path_info = paths_info[key]
                es_instance = esatisfactory_instance(x, epsilon, path_info)
                if estimator.predict(es_instance.reshape(1, -1)) == aim_label:
                    if cost_func(x, es_instance) < delta_mini:
                        x_out = es_instance
                        delta_mini = cost_func(x, es_instance)
            else:
                continue
    return x_out

## How to Use

First of all, we need settings of hyper parameters.

In [22]:
class_labels = [0, 1, 2]  # Same as In [7]
aim_label = 2 # Same as In [7]
epsilon = 0.1

As cost function, I use Euclidean distance.

In [23]:
def cost_func(a, b):
    return np.linalg.norm(a-b)

### Random Forest Prediction

In [24]:
rfc = RandomForestClassifier()
rfc.fit(x_arr, y_arr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Using feature_tweaking()

In [25]:
x = x_arr[0]
x

array([-0.90068117,  1.03205722, -1.3412724 , -1.31297673])

In [26]:
x_new = feature_tweaking(rfc, x, class_labels, aim_label, epsilon, cost_func)
x_new

array([-0.90068117,  1.03205722,  0.77746007, -0.42413893])